[View in Colaboratory](https://colab.research.google.com/github/cahya-wirawan/ML-Collection/blob/master/Text_Classification_BPPT_PANL_TPU.ipynb)

In [0]:
#!pip install textblob
#!pip install -U ntlk
#!pip install torch
#import torch

In [1]:
from sklearn import model_selection, preprocessing
from sklearn import linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import tensorflow as tf
#import pandas as pd, xgboost, numpy, textblob, string
import pandas as pd, xgboost, numpy, string
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers, models, optimizers
from keras.preprocessing.sequence import pad_sequences
from pathlib import Path

import os
import numpy as np
#import ntlk

Using TensorFlow backend.


In [2]:
# Parameters
LMDATA = Path('/content/drive/My Drive/lmdata')
params = {'batch_size': 1024,
          'n_classes': 2,
          'max_len': 100,
          'n_words': 50000,
          'shuffle': True}
try:
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  strategy=tf.contrib.tpu.TPUDistributionStrategy(
      tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
except KeyError:
  TPU_WORKER = None

np.random.seed(seed=10)

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.11.107.34:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 18285324125260461732)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5930766872879097493)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 1297665422713504370)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 190512972810837202)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 15949622976445792891)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
#from google.colab import files

#uploaded = files.upload()

#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [4]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!pwd
print(LMDATA)
!ls -lh "$LMDATA"
!ls -lh "$LMDATA/BPPTIndToEngCorpus"

/content
/content/drive/My Drive/lmdata
total 3.0G
-rw------- 1 root root 4.3M Apr 21 23:28 amazon-review.csv
-rw------- 1 root root  58M Oct  8 16:03 bard-1.h5
-rw------- 1 root root  32M Oct 10 16:21 bard.h5
drwx------ 2 root root 4.0K Oct  9 15:04 BPPTIndToEngCorpus
-rw------- 1 root root  25M Oct  9 16:46 wiki.id.10K.vec
-rw------- 1 root root 755M Oct  9 16:35 wiki.id.300K.vec
-rw------- 1 root root  22M Oct  8 13:31 wiki-news-300d-10K.vec
-rw------- 1 root root 2.2G Mar 14  2018 wiki-news-300d-1M.vec
total 9.2M
-rw------- 1 root root 4.6M Oct  9 15:04 bppt_panl.csv
-rw------- 1 root root 476K Oct  9 15:04 bppt_panl_test.csv
-rw------- 1 root root 4.2M Oct  9 15:04 bppt_panl_train.csv


In [0]:
train_df = pd.read_csv(LMDATA/'BPPTIndToEngCorpus/bppt_panl_train.csv')
train_df.columns = ['label', 'text']
test_df = pd.read_csv(LMDATA/'BPPTIndToEngCorpus/bppt_panl_test.csv')
test_df.columns = ['label', 'text']

In [7]:
print(train_df.head())
#print(train_df['label'][:10].values)
#print(train_df['text'][:10].values)

   label                                               text
0      3  Spurs menguasai permainan sejak saat Keane men...
1      0  Saat ini juga sedang berlangsung negosiasi den...
2      3  Pemain tengah Fenerbahce, Deivid, beralih dari...
3      0  Menurut Kepala Perwakilan BPKP di Provinsi Pap...
4      0  Indonesia memiliki potensi yang luar biasa seb...


In [8]:
#!pip show tensorflow
#!nvidia-smi
!set |grep -i tpu|grep -v grep

COLAB_TPU_ADDR=10.11.107.34:8470
DATALAB_SETTINGS_OVERRIDES='{"datalabBasePath":"/tun/m/tpu-qrllmhg0h7vp/","kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["notebook","-y","--no-browser","--log-level=DEBUG","--debug","--NotebookApp.allow_origin=\"*\"","--NotebookApp.log_format=\"%(message)s\"","--NotebookApp.disable_check_xsrf=True","--NotebookApp.token=","--Session.key=\"\"","--Session.keyfile=\"\"","--ContentsManager.untitled_directory=\"Untitled Folder\"","--ContentsManager.untitled_file=\"Untitled File\"","--ContentsManager.untitled_notebook=\"Untitled Notebook\"","--KernelManager.autorestart=True","--ip=\"172.28.0.2\""]}'


In [0]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_df['text'], train_df['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [10]:
print(train_x[:5])
print(train_y[:5])
print(valid_x[:5])
print(valid_y[:5])


19469    Pada petenis unggulan mendapatkan bye pada per...
24559    Pencetak gol terbanyak Serie A David Trezeguet...
5322     PT Pemeringkat Efek Indonesia Pefindo memberik...
15053    Gas untuk keperluan rumah tangga misalnya, mes...
20938    Kapten Bayern Munich Oliver Kahn memperingatka...
Name: text, dtype: object
[3 3 0 0 3]
7715     Pasukan dari divisi infantri ke tiga yang berm...
12394    Pemerintah dan BI akan tetap menjaga stabilita...
7327     Israel mulai melonggarkan pembatasan di pos pe...
20168    Dua pria tewas dan dua orang lainnya cedera da...
21502    Ia mengatakan meski saat ini banyak arus modal...
Name: text, dtype: object
[1 0 1 1 0]


In [0]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_df['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [0]:
#print(xtrain_count, xvalid_count)


In [0]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                             max_features=5000)
tfidf_vect.fit(train_df['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(train_df['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}',
                                         ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(train_df['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [13]:
# load the pre-trained word-embedding vectors
max_words = params['n_words']
embeddings_index = {}
for i, line in enumerate(open('drive/My Drive/lmdata/wiki.id.10K.vec')):
  if i%10000 == 0:
    print(i)
  values = line.split()
  try:
    embeddings_index[" ".join(values[0:-300])] = numpy.asarray(values[-300:], dtype='float32')
  except ValueError:
    print("Values: {}: {}".format(i, values))

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(train_df['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), 
                                     maxlen=params['max_len'])
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), 
                                     maxlen=params['max_len'])

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if i>=max_words:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

0
10000


In [0]:
train_df['char_count'] = train_df['text'].apply(len)
train_df['word_count'] = train_df['text'].apply(lambda x: len(x.split()))
train_df['word_density'] = train_df['char_count'] / (train_df['word_count']+1)
train_df['punctuation_count'] = train_df['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
train_df['title_word_count'] = train_df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
train_df['upper_case_word_count'] = train_df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [14]:
print(train_df.head())

   label                                               text
0      3  Spurs menguasai permainan sejak saat Keane men...
1      0  Saat ini juga sedang berlangsung negosiasi den...
2      3  Pemain tengah Fenerbahce, Deivid, beralih dari...
3      0  Menurut Kepala Perwakilan BPKP di Provinsi Pap...
4      0  Indonesia memiliki potensi yang luar biasa seb...


In [0]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, 
                is_neural_net=False, epochs=1):
    # fit the training dataset on the classifier
    if is_neural_net:
      classifier.fit(feature_vector_train, label, epochs=epochs)
    else:
      classifier.fit(feature_vector_train, label)   
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
      predictions = [int(round(p[0])) for p in predictions]
      #predictions = predictions.argmax(axis=-1)

    print(" predictions:", predictions[:20])
    print("ground truth:", valid_y[:20])
    
    return metrics.accuracy_score(predictions, valid_y)

In [16]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
NB, Count Vectors:  0.9360279150915964
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
NB, WordLevel TF-IDF:  0.9274498400697877
 predictions: [1 0 1 1 0 0 0 1 2 3 0 1 1 0 0 0 1 2 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
NB, N-Gram Vectors:  0.7916545507414946
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
NB, CharLevel Vectors:  0.8685664437336436


In [25]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print( "LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
LR, Count Vectors:  0.9520209363186973
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
LR, WordLevel TF-IDF:  0.9358825239895319
 predictions: [1 0 1 1 0 0 0 1 2 3 0 1 3 0 3 0 1 0 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
LR, N-Gram Vectors:  0.8269845885431811
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
LR, CharLevel Vectors:  0.9017156150043617


In [26]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)


 predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
SVM, N-Gram Vectors:  0.4287583599883687


In [27]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)

 predictions: [1 0 1 1 0 0 0 1 2 3 0 0 2 0 3 0 1 2 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
RF, Count Vectors:  0.8871765047979063
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 2 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
RF, WordLevel TF-IDF:  0.883396336144228


In [28]:
# Extreme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extreme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

# Extreme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 1 2 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
Xgb, Count Vectors:  0.8200058156440826


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 0 2 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
Xgb, WordLevel TF-IDF:  0.8220412910729863
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
Xgb, CharLevel Vectors:  0.8298924105844723


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
from tensorflow.keras.utils import Sequence, to_categorical

def tokenize(texts, n_words=1000):
    tokenizer = Tokenizer(num_words=n_words)
    tokenizer.fit_on_texts(texts)
    return tokenizer
  
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, texts, labels, tokenizer, batch_size=32, max_len=100,
                 n_classes=2, n_words=1000, shuffle=True):
        'Initialization'
        self.max_len = max_len
        self.batch_size = batch_size
        self.texts = texts
        self.labels = labels
        self.n_classes = n_classes
        self.shuffle = shuffle

        self.tokenizer = tokenizer
        self.steps_per_epoch = int(np.floor(self.texts.size / self.batch_size))
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.steps_per_epoch

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        texts = np.array([self.texts[k] for k in indexes])
        sequences = self.tokenizer.texts_to_sequences(texts)
        X = pad_sequences(sequences, maxlen=self.max_len)
        y = np.array([to_categorical(self.labels[k], 4) for k in indexes])

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        #print(">> on_epoch_end:")
        self.indexes = np.arange(self.texts.size)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [0]:
%%time

def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu", name="D1")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, 
                       is_neural_net=True, epochs=1)
print("NN, Ngram Level TF IDF Vectors",  accuracy)

In [0]:
xtrain_tfidf_ngram.shape

(7500, 5000)

In [0]:
"""
  def to_tpu(func):
    def wrapper(*args, **kwargs):
      print("TPU Wrapper start")
      print(func)
      if TPU_WORKER is not None:
        tpu_model = tf.contrib.tpu.keras_to_tpu_model(func, strategy)
        print("TPU exist")
        return tpu_model(*args, **kwargs)
      else:
        print("TPU not exist")
        return func(*args, **kwargs)
    print("TO_TPU")
    return wrapper
"""

In [0]:
def to_tpu(func):
  if TPU_WORKER is not None:
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(func, strategy)
    return tpu_model
  else:
    return func


In [0]:
def create_cnn(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, 
                                       weights=[embedding_matrix], 
                                       trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.3)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [0]:
# Create data generator
training_generator = DataGenerator(train_x.values, train_y, token, **params)
valid_generator = DataGenerator(valid_x.values, valid_y, token,  **params)


In [31]:
%%time

tf.keras.backend.clear_session()
classifier = create_cnn(input_length=params['max_len'])
#classifier = tf.contrib.tpu.keras_to_tpu_model(classifier, strategy=strategy)
print(classifier)
classifier = to_tpu(classifier)
print(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

classifier.save_weights(str(LMDATA/'bard.h5'), overwrite=True)


TPU Wrapper start
TPU exist
Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 4), dtype=tf.float32, name='dense_1_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Get updates: Tensor("loss_2/mul:0", shape=(), dtype=float32)
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 3.8963499069213867 secs
INFO:tensorflow:Setting weights on TPU model.
19/20 [===========================>..] - ETA: 0s - loss: 1.0140 - acc: 0.6127INFO:tensorflow:New input shapes; (re-)compiling: mode=eval, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 4), dtype=tf.float32, name='dense_1_

In [0]:

classifier.save_weights(str(LMDATA/'bard.h5'), overwrite=True)

In [50]:
# predict the labels on validation dataset


sequences = token.texts_to_sequences(test_df['text'].values)
test_x_seq = pad_sequences(sequences, maxlen=params['max_len'])
#print(valid_x_seq[:5])
classifier = create_cnn(input_length=params['max_len'])
classifier.load_weights(str(LMDATA/'bard.h5'))
"""
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
prediction_model = tf.contrib.tpu.keras_to_tpu_model(
    prediction_model, strategy=strategy)
"""
labels = np.array([list(to_categorical(label, 4).astype(int)) for label in test_df['label'].values])
score = classifier.evaluate(test_x_seq, labels, verbose=1)
print('Loss for final step: {}, accuracy: {}'.format(score[0], score[1]))




3056/3056 [==============================] - 0s 108us/step
Loss for final step: 0.17674767505865135, accuracy: 0.9450261780104712


In [0]:
labels = [list(to_categorical(label, 4).astype(int)) for label in test_df['label'].values]
#print(labels[:5])
print(np.array(labels)[:5])
print(test_df['label'].values)
print(to_categorical(3, 4))

[[0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]
[2 1 0 ... 2 0 2]
[0. 0. 0. 1.]


In [51]:
score = classifier.predict(test_x_seq, verbose=1)
#print(score)
print('Loss for final step: {}'.format(np.argmax(score, axis=1)[:10]))
print(valid_x[:10])

3056/3056 [==============================] - 0s 77us/step
Loss for final step: [2 1 0 0 0 2 2 0 2 0]
7715     Pasukan dari divisi infantri ke tiga yang berm...
12394    Pemerintah dan BI akan tetap menjaga stabilita...
7327     Israel mulai melonggarkan pembatasan di pos pe...
20168    Dua pria tewas dan dua orang lainnya cedera da...
21502    Ia mengatakan meski saat ini banyak arus modal...
19559    Agri Resources BV, sebuah perusahaan di Beland...
6336     Asalkan kita punya program-program yang bagus ...
2881     Sikap ini diperkuat oleh hasil interogasi terh...
16877            Gurita juga menggunakan sistem yang sama.
26868    Tantangan utamanya mungkin akan datang dari Ra...
Name: text, dtype: object


In [0]:
!free

              total        used        free      shared  buff/cache   available
Mem:       13335236     1705600     3564096         804     8065540    12425368
Swap:             0           0           0


In [0]:

#print('Accuracy ', score[1])
"""
# First, run the seed forward to prime the state of the model.
#prediction_model.reset_states()
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
prediction_model = tf.contrib.tpu.keras_to_tpu_model(
    prediction_model, strategy=strategy)

predictions = prediction_model.predict(valid_x_seq)

print("predictions", predictions[:20])

predictions = tpu_model.predict(valid_x_seq)

predictions = [int(round(p[0])) for p in predictions]
#predictions = predictions.argmax(axis=-1)

print("predictions", predictions[:20])
print("valid_y", valid_y[:20])

return metrics.accuracy_score(predictions, valid_y)
"""

In [0]:
print(score)
print(np.argmax(score, axis=1))


[[3.3721889e-04 1.7559213e-04 9.9948490e-01 2.2181157e-06]
 [1.0414308e-08 9.9999595e-01 2.9743585e-06 1.0576000e-06]
 [9.7819394e-01 6.5141567e-03 1.5249715e-02 4.2119769e-05]
 ...
 [1.3407861e-02 1.9714341e-02 9.6600837e-01 8.6940167e-04]
 [9.9999952e-01 3.1176335e-09 4.6144635e-07 4.8664687e-11]
 [2.9894806e-04 7.0751207e-07 9.9969995e-01 4.3038932e-07]]
[2 1 0 ... 2 0 2]


# New Section

In [0]:
def create_rnn_lstm(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.2)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)
    #lstm_layer = layers.LSTM(100)(lstm_layer)
    #lstm_layer = layers.LSTM(100, return_sequences=True)(lstm_layer)
    #lstm_layer = layers.TimeDistributed(layers.Dense(100))(lstm_layer)
    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.2)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [54]:
%%time

tf.keras.backend.clear_session()
classifier = create_rnn_lstm(input_length=params['max_len'])
#classifier = tf.contrib.tpu.keras_to_tpu_model(classifier, strategy=strategy)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
20/20 [==============================] - 12s 608ms/step - loss: 1.0240 - acc: 0.5697 - val_loss: 0.5509 - val_acc: 0.8366
Epoch 2/20
20/20 [==============================] - 9s 439ms/step - loss: 0.4237 - acc: 0.8557 - val_loss: 0.2883 - val_acc: 0.8965
Epoch 3/20
20/20 [==============================] - 9s 444ms/step - loss: 0.3061 - acc: 0.8934 - val_loss: 0.2479 - val_acc: 0.9118
Epoch 4/20
20/20 [==============================] - 9s 435ms/step - loss: 0.2696 - acc: 0.9048 - val_loss: 0.2386 - val_acc: 0.9149
Epoch 5/20
20/20 [==============================] - 9s 440ms/step - loss: 0.2481 - acc: 0.9127 - val_loss: 0.2233 - val_acc: 0.9219
Epoch 6/20
20/20 [==============================] - 9s 444ms/step - loss: 0.2391 - acc: 0.9166 - val_loss: 0.2154 - val_acc: 0.9271
Epoch 7/20
20/20 [==============================] - 9s 446ms/step - loss: 0.2238 - acc: 0.9207 - val_loss: 0.2110 - val_acc: 0.9251
Epoch 8/20
20/20 [==============================] - 9s 444ms/step - loss: 0

In [0]:
%%time

def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=10)
print("RNN-GRU, Word Embeddings",  accuracy)

ValueError: ignored

In [0]:
%%time

def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=1)
print("RNN-Bidirectional, Word Embeddings",  accuracy)

Epoch 1/1
7500/7500 [==============================] - 38s 5ms/step - loss: 0.6029
predictions [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
RNN-Bidirectional, Word Embeddings 0.7936
CPU times: user 1min 21s, sys: 1.16 s, total: 1min 22s
Wall time: 45.7 s


In [0]:
%%time

def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=10)
print("CNN, Word Embeddings",  accuracy)

Epoch 1/10
7500/7500 [==============================] - 12s 2ms/step - loss: 0.5678
Epoch 2/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.3829
Epoch 3/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.3374
Epoch 4/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.2814
Epoch 5/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.2365
Epoch 6/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.2043
Epoch 7/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1675
Epoch 8/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1320
Epoch 9/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1081
Epoch 10/10
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1005
predictions [0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]
valid_y [0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0]
CNN, Word Embeddings 0.8624
CPU time

# New Section